In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')
##################################    
   

def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if  token.text == '\n' or token.text.startswith('\n'):
            doc[token.i+1].is_sent_start = True
    return doc

nlp.add_pipe(set_custom_boundaries, before='parser')

In [3]:

df = pd.read_csv('P&P training-utf8.csv',encoding="utf-8")

print(len(df))
df.head()


285


,number,short_description,Class,description
0,INC0809260,Y2684992,P,From: Rebecca Connolly <Rebecca.Connolly@nbrow...
1,INC0808684,Delivery offer not working,P,NaN
2,INC0808628,Delivery pass needs removing on Staff Accounts...,R,Please can you remove the delivery pass be rem...
3,INC0807303,G7121564,O,please set (ACUSTRSS ) to 1 so account holder ...
4,INC0807253,Delivery pass needs removing on Staff Accounts...,R,Please can the delivery pass be removed from t...


In [4]:
# REMOVE NaN VALUES AND EMPTY STRINGS:
df.dropna(inplace=True)

blanks = []  # start with an empty list`

for i,n,sd,c,ds in df.itertuples():  # iterate over the DataFrame
    if type(ds)==str:            # avoid NaN values
        if ds.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list


In [5]:
df.drop(blanks, inplace=True)

In [6]:
raw = df['description'][0]
ra = raw.lower()
txt_tokens = nlp(ra)
all_stopwords = nlp.Defaults.stop_words
tokens_without_sw= [word.text for word in txt_tokens if not word.text in all_stopwords]
stri = ' '.join(tokens_without_sw)
stri

': rebecca connolly < rebecca.connolly@nbrown.co.uk > \n sent : 26 2020 01:25 pm \n : serviceline < serviceline@jdwilliams.co.uk > \n subject : y2684992 \n \n good afternoon , \n\n customer charged delivery purchasing subscription . needs activating ? \n\n thank advance , \n\n rebecca connolly \n\n proactive notification team \n\n mobile : + 44 ( 0)773 990 2504 \n reception : + 44 ( 0)161 236 8256 \n\n \n\n\n\n\n\n griffin house \n\n 40 lever st \n manchester \n m60 6es \n www.nbrown.co.uk \n\n \n\n \n\n n brown 10 uk clothing & footwear digital retailer . key retail brands jd williams , simply , jacamo ambrose wilson . \n\n \n\n'

In [9]:
cleaned_df = pd.DataFrame()
for i,n,sd,c,ds in df.itertuples():
 #   print(i, n, sd, c, ds)   
    sd = sd.lower()
    cleaned_df = cleaned_df.append({'inc_num': n, 'short_des': sd, 'class': c, 'desc': reg_func(clean_desc(ds))}, ignore_index=True)
len(cleaned_df)    

253

In [7]:

def clean_desc(data):
#     import email
    
#     msg = email.message_from_string(abc)

#     for part in msg.walk():
#         # each part is a either non-multipart, or another multipart message
#         # that contains further parts... Message is organized like a tree
#         if part.get_content_type() == 'text/plain':

#             body = part.get_payload(decode=False)

    ##################################
    from spacy.matcher import PhraseMatcher
    matcher = PhraseMatcher(nlp.vocab)
    phrase_list = ['Griffin House', '40 Lever St', 'M60 6ES','Sent:','To:','Subject:','From:']
    phrase_patterns = [nlp(text) for text in phrase_list]
    matcher.add('To:',None,*phrase_patterns) 
    unwanted = []

    
    doc2 = nlp(data)
    sents = [sent for sent in doc2.sents]

    for x in sents:
        
         matches = matcher(nlp(str(x)))
         if len(matches) > 0:
            unwanted.append(x)
        
    sents =[ele for ele in sents if ele not in unwanted]
    
    return sents




In [8]:
###################################################
def reg_func(sents):
    import re
    txt_patterns = r'\d{3} \d{3} \d{4}|\d{3}-\d{3}-\d{4}|\d{11}|\d{10}'

    exclude = []
    found_flag = False
    for txt in sents:
        matches_txt = re.findall(txt_patterns,str(txt))
        #print(f'first entry into sents, value of flag = {found_flag}', len(matches_txt))
        if len(matches_txt) > 0 or found_flag == True:
            found_flag = True
            exclude.append(txt)
    
    sents =[ele for ele in sents if ele not in exclude]
    list_of_strings  = [i.text for i in sents]
    Penul_desc = ''.join(list_of_strings)
    
#     txt_tokens = nlp(Penul_desc)
#     tokens_without_sw= [word.text for word in txt_tokens if not word.text in all_stopwords]
#     Final_desc = ' '.join(tokens_without_sw)
    
#     Final_desc = Final_desc.lower()
    Penul_desc = Penul_desc.lower()
    return Penul_desc

In [42]:
def remove_sw(Penul_desc):
    txt_tokens = nlp(Penul_desc)
    tokens_without_sw= [word.text for word in txt_tokens if not word.text in all_stopwords]
    Final_desc = ' '.join(tokens_without_sw)

In [10]:
cleaned_df['desc'][0]

'good afternoon,\n\nthis customer is being charged for delivery after purchasing the subscription.please can you see if it needs activating?\n\nthank you in advance,\n\nrebecca connolly\n\nproactive notification team\n\n'

In [11]:
cleaned_df.to_csv(r'Cleaned-P&P data.csv', index = False)